In [1]:
import pandas as pd

movieid_title_genres = pd.read_csv("movies.csv")
movieid_title_genres['genres'] = movieid_title_genres['genres'].apply(lambda x: x.split('|'))
userId_movieId_rating = pd.read_csv("train_ratings_binary.csv")
movieid_genres = movieid_title_genres.drop(columns=['title'])
movieid_genres.head()

,movieId,genres
0,1,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,"[Adventure, Children, Fantasy]"
2,3,"[Comedy, Romance]"
3,4,"[Comedy, Drama, Romance]"
4,5,[Comedy]


In [2]:
movieid_title_genres.head()
#userId_movieid_genres = pd.merge(movieidc_genres, right, on=['key1', 'key2'])

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [3]:
userId_movieid_genres = pd.merge(movieid_genres, userId_movieId_rating, on=['movieId'], how='right')
userId_movieid_genres.head()

,movieId,genres,userId,rating
0,1,"[Adventure, Animation, Children, Comedy, Fantasy]",2,1
1,1,"[Adventure, Animation, Children, Comedy, Fantasy]",3,1
2,1,"[Adventure, Animation, Children, Comedy, Fantasy]",6,1
3,1,"[Adventure, Animation, Children, Comedy, Fantasy]",7,1
4,1,"[Adventure, Animation, Children, Comedy, Fantasy]",10,1


In [4]:
user_genre_rating = {}
for index, row in userId_movieid_genres.iterrows():
    if row['rating']==0:
        continue
    elif row['userId'] not in user_genre_rating:
        user_genre_rating[row['userId']] = {}
        for genre in row['genres']:
            user_genre_rating[row['userId']][genre] = 1
    else:
        for genre in row['genres']:
            if genre in user_genre_rating[row['userId']]:
                user_genre_rating[row['userId']][genre] = user_genre_rating[row['userId']][genre]+1
            else:
                user_genre_rating[row['userId']][genre] = 1


In [5]:
user_genre_rating[1]

{'Action': 6,
 'Drama': 5,
 'Romance': 2,
 'War': 2,
 'Adventure': 3,
 'Sci-Fi': 4,
 'Crime': 1,
 'Mystery': 1,
 'Western': 1}

In [6]:
for userid in user_genre_rating:
    user_genre_rating[userid]["total"] = 0
    for genre in user_genre_rating[userid]:
        if genre != "total":
            user_genre_rating[userid]["total"] += user_genre_rating[userid][genre]
    for genre in user_genre_rating[userid]:
        if genre != "total":
            user_genre_rating[userid][genre] /= user_genre_rating[userid]["total"]
    del user_genre_rating[userid]['total']

In [7]:
user_genre_rating[1]

{'Action': 0.24,
 'Drama': 0.2,
 'Romance': 0.08,
 'War': 0.08,
 'Adventure': 0.12,
 'Sci-Fi': 0.16,
 'Crime': 0.04,
 'Mystery': 0.04,
 'Western': 0.04}

In [8]:
val_data = pd.read_csv("val_ratings_binary.csv")
val_data["Id"] = val_data.index
val_data.head()

,userId,movieId,rating,Id
0,1,1653.0,1,0
1,1,2011.0,0,1
2,1,2054.0,0,2
3,1,2406.0,1,3
4,1,2571.0,1,4


In [9]:
val_data_genres = pd.merge(val_data, movieid_genres, on=['movieId'])
val_data_genres = val_data_genres.sort_values(by=['Id'])
val_data_genres.head()

,userId,movieId,rating,Id,genres
0,1,1653.0,1,0,"[Drama, Sci-Fi, Thriller]"
1309,1,2011.0,0,1,"[Adventure, Comedy, Sci-Fi]"
3497,1,2054.0,0,2,"[Adventure, Children, Comedy, Fantasy, Sci-Fi]"
5915,1,2406.0,1,3,"[Action, Adventure, Comedy, Romance]"
10492,1,2571.0,1,4,"[Action, Sci-Fi, Thriller]"


In [10]:
f = lambda x: sum([ 0 if x['userId'] not in user_genre_rating or genre not in user_genre_rating[x['userId']] else user_genre_rating[x['userId']][genre] for genre in x['genres'] ])/len(x['genres'])
val_data_genres['predicted'] = val_data_genres.apply(f, axis=1)
val_data_genres.head()

,userId,movieId,rating,Id,genres,predicted
0,1,1653.0,1,0,"[Drama, Sci-Fi, Thriller]",0.120000
1309,1,2011.0,0,1,"[Adventure, Comedy, Sci-Fi]",0.093333
3497,1,2054.0,0,2,"[Adventure, Children, Comedy, Fantasy, Sci-Fi]",0.056000
5915,1,2406.0,1,3,"[Action, Adventure, Comedy, Romance]",0.110000
10492,1,2571.0,1,4,"[Action, Sci-Fi, Thriller]",0.133333


In [17]:
val_data_genres[val_data_genres['predicted']>0.1].shape[0] / val_data_genres.shape[0]

0.5752718769284934

In [18]:
normalized_rating_val = val_data_genres.copy()
normalized_rating_val['predicted'] = normalized_rating_val['predicted'].apply(lambda x: 1 if x>=0.1 else 0)
normalized_rating_val.head()

,userId,movieId,rating,Id,genres,predicted
0,1,1653.0,1,0,"[Drama, Sci-Fi, Thriller]",1
1309,1,2011.0,0,1,"[Adventure, Comedy, Sci-Fi]",0
3497,1,2054.0,0,2,"[Adventure, Children, Comedy, Fantasy, Sci-Fi]",0
5915,1,2406.0,1,3,"[Action, Adventure, Comedy, Romance]",1
10492,1,2571.0,1,4,"[Action, Sci-Fi, Thriller]",1


In [21]:
'''
numcorrect = 0
for index, row in normalized_rating_val.iterrows():
    if row['predicted']==row['rating']:
        numcorrect += 1
numcorrect/val_data_genres.shape[0]
'''
normalized_rating_val['correct'] = normalized_rating_val.apply(lambda x:1 if x['predicted']==x['rating'] else 0, axis=1)


2132349

In [22]:
sum(normalized_rating_val['correct'])/normalized_rating_val.shape[0]

0.5331890891160211

In [80]:
test_data = pd.read_csv("test_ratings.csv")
test_data["Id"] = test_data.index
test_data.head()

,userId,movieId,Id
0,1,4993.0,0
1,1,5445.0,1
2,1,5952.0,2
3,1,7153.0,3
4,1,8636.0,4


In [81]:
test_userId_movieid_genres = pd.merge(test_data, movieid_genres, on=['movieId'])
test_userId_movieid_genres = test_userId_movieid_genres.sort_values(by=['Id'])
test_userId_movieid_genres.head()

,userId,movieId,Id,genres
0,1,4993.0,0,"[Adventure, Fantasy]"
10296,1,5445.0,1,"[Action, Crime, Mystery, Sci-Fi, Thriller]"
18371,1,5952.0,2,"[Adventure, Fantasy]"
31220,1,7153.0,3,"[Action, Adventure, Drama, Fantasy]"
46924,1,8636.0,4,"[Action, Adventure, Sci-Fi, IMAX]"


In [33]:
'''
import statistics

#statistics.median([1,2,3,5])
for userid in user_genre_rating:
    del user_genre_rating[userid]['total']
    mid = statistics.median(list(user_genre_rating[userid].values()))
    for genre in user_genre_rating[userid]:
        if user_genre_rating[userid][genre] >= mid:
            user_genre_rating[userid][genre] = 1
        else:
            user_genre_rating[userid][genre] = 0
user_genre_rating[1]
'''

{'Action': 1,
 'Drama': 1,
 'Romance': 1,
 'War': 1,
 'Adventure': 1,
 'Sci-Fi': 1,
 'Crime': 0,
 'Mystery': 0,
 'Western': 0}

In [82]:
f = lambda x: sum([ 0 if x['userId'] not in user_genre_rating or genre not in user_genre_rating[x['userId']] else user_genre_rating[x['userId']][genre] for genre in x['genres'] ])/len(x['genres'])
test_userId_movieid_genres['rating'] = test_userId_movieid_genres.apply(f, axis=1)
test_userId_movieid_genres.head()

,userId,movieId,Id,genres,rating
0,1,4993.0,0,"[Adventure, Fantasy]",0.50
10296,1,5445.0,1,"[Action, Crime, Mystery, Sci-Fi, Thriller]",0.40
18371,1,5952.0,2,"[Adventure, Fantasy]",0.50
31220,1,7153.0,3,"[Action, Adventure, Drama, Fantasy]",0.75
46924,1,8636.0,4,"[Action, Adventure, Sci-Fi, IMAX]",0.75


In [89]:
normalized_rating = test_userId_movieid_genres.copy()
normalized_rating['rating'] = normalized_rating['rating'].apply(lambda x: 1 if x>=0.65 else 0)

In [90]:
normalized_rating = normalized_rating.drop(columns=['userId', 'movieId', 'genres'])
normalized_rating.head()

,Id,rating
0,0,0
10296,1,0
18371,2,0
31220,3,1
46924,4,1


In [91]:
normalized_rating.to_csv("user_genre_prediction.csv", index=False)